In [0]:
# Chicago Taxi – Gold Analytics (DBCE)

Purpose: Produce business-oriented aggregates from silver into a Delta gold table + UC VIEW.

Exam coverage:
- Transformations & aggregations (PySpark/SQL).
- Medallion: silver → gold, read/write Delta by path.
- Validation & simple performance considerations (cache/explain notes).


In [0]:
# Core imports for gold analytics
from pyspark.sql.functions import col, sum as _sum, countDistinct, date_trunc, count, avg 
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


In [0]:
# Notebook parameters for pipeline execution
dbutils.widgets.text("catalog", "taxi_catalog")
dbutils.widgets.text("schema", "taxi_schema")
dbutils.widgets.text("volume", "taxi_volume")

# Read widget values
catalog_name = dbutils.widgets.get("catalog")
schema_name = dbutils.widgets.get("schema")
volume_name = dbutils.widgets.get("volume")

# Base paths
base_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}"
silver_path = f"{base_path}/silver"
gold_path = f"{base_path}/gold"

# Entities
silver_table_path = f"{silver_path}/chicago_taxi_trips_silver"
gold_table_path = f"{gold_path}/chicago_taxi_kpis"
gold_view_name = f"{catalog_name}.{schema_name}.chicago_taxi_gold_v"


In [0]:
# Read silver
silver_df = spark.read.format("delta").load(silver_table_path)

# Build aggregates
gold_df = (
    silver_df
    .withColumn("month", date_trunc("month", col("trip_start_timestamp")))
    .groupBy("month", "payment_type")
    .agg(
        _sum(col("trip_total")).alias("total_revenue"),
        count("*").alias("total_trips"),
        avg(col("fare")).alias("avg_fare"),
        countDistinct("company").alias("distinct_companies")
    )
)


# Write Delta (path)
(
    gold_df
    .write
    .format("delta")
    .mode("overwrite")
    .save(gold_table_path)
)

# Expose UC VIEW
spark.sql(f"""
    CREATE OR REPLACE VIEW {gold_view_name} AS
    SELECT * FROM delta.`{gold_table_path}`
""")

# Consistency checks: path vs view
cnt_path = spark.read.format("delta").load(gold_table_path).count()
cnt_view = spark.table(gold_view_name).count()
assert cnt_path == cnt_view and cnt_path > 0, "Gold is empty or mismatched."

# Optional sanity: total_revenue should be non_negative
neg_rev = spark.sql(f"""
    SELECT SUM(CASE WHEN total_revenue < 0 THEN 1 ELSE 0 END) as neg_rows
    FROM {gold_view_name}
""").collect()[0][0]
assert neg_rev == 0, "Negative total_revenue foun in gold"

print("Gold written at:", gold_table_path)
print("Gold view", gold_view_name, "| rows:", cnt_view)

In [0]:
# Inspect pysical plan of the gold aggregation and a simple filtered read from the view
from pyspark.sql.functions import col

print("==  DataFrame plan for gold_df ==")
gold_df.explain(True)

print("\n==  Plan for reading the gold view with a filder ==")
spark.table(gold_view_name).where(col("total_revenue") > 0).limit(1).explain(True)